<a href="https://colab.research.google.com/github/Mostafa-eltalawy/Question-Generation/blob/master/question_generation(18_3_2021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -U transformers==3.0.0
#!pip install PIL
#!pip install pytesseract
#!pip install pdf2image
#!sudo apt-get install tesseract-ocr

In [ ]:
#!python -m nltk.downloader punkt
#!pip install allennlp

In [ ]:
import itertools
import logging
from typing import Optional, Dict, Union

from nltk import sent_tokenize

import torch
from transformers import(
    AutoModelForSeq2SeqLM, 
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,)

from transformers import AutoModelWithLMHead, AutoTokenizer
logger = logging.getLogger(__name__)
# Importing package and summarizer
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import spacy
import re

import spacy
from spacy.matcher import Matcher 

from spacy import displacy 
#import visualise_spacy_tree
from IPython.display import Image, display

# load english language model
nlp = spacy.load('en_core_web_sm')
# Import libraries for pdf reading
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

from collections import Counter
import en_core_web_sm

In [ ]:
#reading pdf file and converting it to text file by OCR
# Path of the pdf 
PDF_file = "Invest-Law.pdf"

''' 
Part #1 : Converting PDF to images 
'''
# Store all the pages of the PDF in a variable 
pages = convert_from_path(PDF_file, 500) 
# Counter to store images of each page of PDF to image 
image_counter = 1
# Iterate through all the pages stored above 
for page in pages: 
	filename = "page_"+str(image_counter)+".jpg"
	# Save the image of the page in system 
	page.save(filename, 'JPEG') 
	# Increment the counter to update filename 
	image_counter = image_counter + 1

''' 
Part #2 - Recognizing text from the images using OCR 
'''
# Variable to get count of total number of pages 
filelimit = image_counter-1
# Creating a text file to write the output 
outfile = "out_text.txt"
# Open the file in append mode so that All contents of all images are added to the same file 
textfile = open(outfile, "a") 
# Iterate from 1 to total number of pages 
for i in range(1, filelimit + 1): 
	filename = "page_"+str(i)+".jpg"	
	# Recognize the text as string in image using pytesserct 
	text = str(((pytesseract.image_to_string(Image.open(filename))))) 
	text = text.replace('-\n', '')	 
	# Finally, write the processed text to the file. 
	textfile.write(text) 
textfile.close() 

f = open("out_text", "r")
inputtext = f.read()

In [ ]:
class Question_Generation_Pipeline:
    """Poor man's QG pipeline"""
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        ans_model: PreTrainedModel,
        ans_tokenizer: PreTrainedTokenizer,
        qg_format: str,
        use_cuda: bool
    ):
        self.model = model
        self.tokenizer = tokenizer

        self.ans_model = ans_model
        self.ans_tokenizer = ans_tokenizer

        self.qg_format = qg_format

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        if self.ans_model is not self.model:
            self.ans_model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"

    def __call__(self, inputs: str):
        inputs = " ".join(inputs.split())
        sents, answers = self._extract_answers(inputs)
        flat_answers = list(itertools.chain(*answers))
        
        if len(flat_answers) == 0:
          return []

        if self.qg_format == "prepend":
            qg_examples = self._prepare_inputs_for_qg_from_answers_prepend(inputs, answers)
        else:
            qg_examples = self._prepare_inputs_for_qg_from_answers_hl(sents, answers)
        
        qg_inputs = [example['source_text'] for example in qg_examples]
        questions = self._generate_questions(qg_inputs)
        output = [{'answer': example['answer'], 'question': que} for example, que in zip(qg_examples, questions)]
        return output
    
    def _generate_questions(self, inputs):
        inputs = self._tokenize(inputs, padding=True, truncation=True)
        
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
            num_beams=4,
        )
        
        questions = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
        return questions
    
    def _extract_answers(self, context):
        sents, inputs = self._prepare_inputs_for_ans_extraction(context)
        inputs = self._tokenize(inputs, padding=True, truncation=True)

        outs = self.ans_model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
        )
        
        dec = [self.ans_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
        answers = [item.split('<sep>') for item in dec]
        answers = [i[:-1] for i in answers]
        
        return sents, answers
    
    def _tokenize(self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512 
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs
    
    def _prepare_inputs_for_ans_extraction(self, text):
        sents = sent_tokenize(text)

        inputs = []
        for i in range(len(sents)):
            source_text = "extract answers:"
            for j, sent in enumerate(sents):
                if i == j:
                    sent = "<hl> %s <hl>" % sent
                source_text = "%s %s" % (source_text, sent)
                source_text = source_text.strip()
            
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            inputs.append(source_text)

        return sents, inputs
    
    def _prepare_inputs_for_qg_from_answers_hl(self, sents, answers):
        inputs = []
        for i, answer in enumerate(answers):
            if len(answer) == 0: continue
            for answer_text in answer:
                sent = sents[i]
                sents_copy = sents[:]
                
                answer_text = answer_text.strip()
                
                ans_start_idx = sent.index(answer_text)
                
                sent = f"{sent[:ans_start_idx]} <hl> {answer_text} <hl> {sent[ans_start_idx + len(answer_text): ]}"
                sents_copy[i] = sent
                
                source_text = " ".join(sents_copy)
                source_text = f"generate question: {source_text}" 
                if self.model_type == "t5":
                    source_text = source_text + " </s>"
                
                inputs.append({"answer": answer_text, "source_text": source_text})
        
        return inputs
    
    def _prepare_inputs_for_qg_from_answers_prepend(self, context, answers):
        flat_answers = list(itertools.chain(*answers))
        examples = []
        for answer in flat_answers:
            source_text = f"answer: {answer} context: {context}"
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            
            examples.append({"answer": answer, "source_text": source_text})
        return examples

In [ ]:
class MultiTask_QAQG_Pipeline(Question_Generation_Pipeline):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def __call__(self, inputs: Union[Dict, str]):
        if type(inputs) is str:
            # do qg
            return super().__call__(inputs)
        else:
            # do qa
            return self._extract_answer(inputs["question"], inputs["context"])
    
    def _prepare_inputs_for_qa(self, question, context):
        source_text = f"question: {question}  context: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        return  source_text
    
    def _extract_answer(self, question, context):
        source_text = self._prepare_inputs_for_qa(question, context)
        inputs = self._tokenize([source_text], padding=False)
    
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=16, #changed from 16 to 10 
        )

        answer = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        return answer

In [ ]:
class E2E_Question_Generation_Pipeline:
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        use_cuda: bool
    ) :

        self.model = model
        self.tokenizer = tokenizer

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"
        
        self.default_generate_kwargs = {
            "max_length": 256, #changed from 256 to 10
            "num_beams": 4,
            "length_penalty": 1.5,
            "no_repeat_ngram_size": 3,
            "early_stopping": True,
        }
    
    def __call__(self, context: str, **generate_kwargs):
        inputs = self._prepare_inputs_for_e2e_qg(context)

        # TODO: when overrding default_generate_kwargs all other arguments need to be passsed
        # find a better way to do this
        if not generate_kwargs:
            generate_kwargs = self.default_generate_kwargs
        
        input_length = inputs["input_ids"].shape[-1]
        
        # max_length = generate_kwargs.get("max_length", 256)
        # if input_length < max_length:
        #     logger.warning(
        #         "Your max_length is set to {}, but you input_length is only {}. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)".format(
        #             max_length, input_length
        #         )
        #     )

        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device),
            **generate_kwargs
        )

        prediction = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        questions = prediction.split("<sep>")
        questions = [question.strip() for question in questions[:-1]]
        return questions
    
    def _prepare_inputs_for_e2e_qg(self, context):
        source_text = f"generate questions: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        
        inputs = self._tokenize([source_text], padding=False)
        return inputs
    
    def _tokenize(
        self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs

In [ ]:
modelPath_e2e_qg_small='valhalla/t5-small-e2e-qg/'  # valhalla folder must be at the same directory with the code
modelPath_qg_small='valhalla/t5-small-qg-hl/'
modelPath_qa_qg_small='valhalla/t5-small-qa-qg-hl/'

modelPath_e2e_qg_base='valhalla/t5-base-e2e-qg/'
modelPath_qg_base='valhalla/t5-base-qg-hl/'
modelPath_qa_qg_base='valhalla/t5-base-qa-qg-hl/'



In [ ]:
#downloading and saving model files one time only
model.save_pretrained(modelPath_e2e_qg_small)  #downloading model and save it to folder valhalla then folder t5-small-e2e-qg
model.save_pretrained(modelPath_qg_small)  #downloading model and save it to folder valhalla then folder t5-small-qg-hl
model.save_pretrained(modelPath_qa_qg_small)  #downloading model and save it to folder valhalla then folder t5-small-qa-qg-hl
model.save_pretrained(modelPath_e2e_qg_base)  #downloading model and save it to folder valhalla then folder t5-base-e2e-qg
model.save_pretrained(modelPath_qg_base)  #downloading model and save it to folder valhalla then folder t5-base-qg-hl
model.save_pretrained(modelPath_qa_qg_base)  #downloading model and save it to folder valhalla then folder t5-base-qa-qg-hl

In [ ]:
#tokenizer.save_pretrained(modelPath_e2e_qg)   #downloading model and save it to folder valhalla then folder t5-small-e2e-qg
#tokenizer = AutoTokenizer.from_pretrained(modelPath_qa_qg_small)
#model = AutoModelForSeq2SeqLM.from_pretrained(modelPath_qa_qg_small)

In [ ]:
SUPPORTED_TASKS = {
    "question-generation": {
        "impl": Question_Generation_Pipeline,
        "default": {
            "model": modelPath_qg_small,
            "ans_model": modelPath_qa_qg_small,
        }
    },
    "multitask-qa-qg": {
        "impl": MultiTask_QAQG_Pipeline,
        "default": {
            "model": modelPath_qa_qg_small,
        }
    },
    "e2e-qg": {
        "impl": E2E_Question_Generation_Pipeline,
        "default": {
            "model": modelPath_e2e_qg_small,
        }
    }
}

In [ ]:
def GenAnsPipline(
    task: str,
    model: Optional = None,
    tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    qg_format: Optional[str] = "highlight",
    ans_model: Optional = None,
    ans_tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    use_cuda: Optional[bool] = True,
    **kwargs,
):
    # Retrieve the task
    if task not in SUPPORTED_TASKS:
        raise KeyError("Unknown task {}, available tasks are {}".format(task, list(SUPPORTED_TASKS.keys())))

    targeted_task = SUPPORTED_TASKS[task]
    task_class = targeted_task["impl"]

    # Use default model/config/tokenizer for the task if no model is provided
    if model is None:
        model = targeted_task["default"]["model"]
    
    # Try to infer tokenizer from model or config name (if provided as str)
    if tokenizer is None:
        if isinstance(model, str):
            tokenizer = model
        else:
            # Impossible to guest what is the right tokenizer here
            raise Exception(
                "Impossible to guess which tokenizer to use. "
                "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
            )
    
    # Instantiate tokenizer if needed
    if isinstance(tokenizer, (str, tuple)):
        if isinstance(tokenizer, tuple):
            # For tuple we have (tokenizer name, {kwargs})
            tokenizer = AutoTokenizer.from_pretrained(tokenizer[0], **tokenizer[1])
        else:
            tokenizer = AutoTokenizer.from_pretrained(tokenizer)
    
    # Instantiate model if needed
    if isinstance(model, str):
        model = AutoModelForSeq2SeqLM.from_pretrained(model)
    
    if task == "question-generation":
        if ans_model is None:
            # load default ans model
            ans_model = targeted_task["default"]["ans_model"]
            ans_tokenizer = AutoTokenizer.from_pretrained(ans_model)
            ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
        else:
            # Try to infer tokenizer from model or config name (if provided as str)
            if ans_tokenizer is None:
                if isinstance(ans_model, str):
                    ans_tokenizer = ans_model
                else:
                    # Impossible to guest what is the right tokenizer here
                    raise Exception(
                        "Impossible to guess which tokenizer to use. "
                        "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
                    )
            
            # Instantiate tokenizer if needed
            if isinstance(ans_tokenizer, (str, tuple)):
                if isinstance(ans_tokenizer, tuple):
                    # For tuple we have (tokenizer name, {kwargs})
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer[0], **ans_tokenizer[1])
                else:
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer)

            if isinstance(ans_model, str):
                ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
    
    if task == "e2e-qg":
        return task_class(model=model, tokenizer=tokenizer, use_cuda=use_cuda)
    elif task == "question-generation":
        return task_class(model=model, tokenizer=tokenizer, ans_model=ans_model, ans_tokenizer=ans_tokenizer, qg_format=qg_format, use_cuda=use_cuda)
    else:
        return task_class(model=model, tokenizer=tokenizer, ans_model=model, ans_tokenizer=tokenizer, qg_format=qg_format, use_cuda=use_cuda)


# qenerating and answering task loading

In [ ]:
#generating Questions using model="valhalla/t5-small-qg-hl"
QGSmall = GenAnsPipline("question-generation")

In [ ]:
#generating Questions using model="valhalla/t5-base-qg-hl"
QGBase = GenAnsPipline("question-generation", model=modelPath_qg_base)

In [ ]:
#generating Questions and answering  using model="valhalla/t5-small-qa-qg-hl"
MultiTaskQAQGsmall = GenAnsPipline("multitask-qa-qg")

In [ ]:
#generating Questions and answering  using model="valhalla/t5-base-qa-qg-hl"
MultiTaskQAQGbase = GenAnsPipline("multitask-qa-qg", model=modelPath_qa_qg_base)

In [ ]:
#generating Questions using model="valhalla/t5-small-e2e-qg"
E2EQGsmall = GenAnsPipline("e2e-qg")

In [ ]:
#generating Questions using model="valhalla/t5-base-e2e-qg"
E2EQGbase = GenAnsPipline("e2e-qg", model=modelPath_e2e_qg_base)

In [ ]:
# function to clean text
def clean(text):
    # removing paragraph numbers
    #text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\\[]*[\\]]", "", str(text))
    
    return text

In [ ]:
# split inputtext to several sentences 
def sentences(text):
  text = re.split('[.?]', text)
  clean_sent = []
  for sent in text:
      clean_sent.append(sent)
  return clean_sent

In [ ]:
def GetQuestionsAndAnswers(inputtext):
  cleanedtext=clean(inputtext)
  listedtext=sentences(inputtext)
  questions1=E2EQGsmall(listedtext)
  questions2=E2EQGbase(listedtext)
  #print('questions list is : \n' , questions)
  #print('====================================================================')
  for question in questions1:
    answer1 = MultiTaskQAQGsmall({"question": question, "context": cleanedtext})
    answer2 = MultiTaskQAQGbase({"question": question, "context": cleanedtext})
    print( ' question (small model) is : ' ,question  )
    print( ' answer1 (small model) is : ' ,answer1  )
    print( ' answer2 (base model) is : ' ,answer2  )
    print('------------------------------------------------------------------')
  print('=====================================================================================')
  for question in questions2:
    answer1 = MultiTaskQAQGsmall({"question": question, "context": cleanedtext})
    answer2 = MultiTaskQAQGbase({"question": question, "context": cleanedtext})
    print( ' question (base model) is : ' ,question  )
    print( ' answer1  (small model) is : ' ,answer1  )
    print( ' answer2 (base model) is : ' ,answer2  )
    print('------------------------------------------------------------------')
  #return questions , answer


In [ ]:
def getkeywordsEntSum(inputtext):
  textKeywords=keywords(inputtext, words=20, lemmatize=True)
  print(textKeywords)
  summerizedtext=summarize(inputtext, ratio=0.6)
  print(summerizedtext)
  doc=nlp(inputtext)
  for ent in doc.ents: 
    print(ent.text, ' .. and word type is ..',  ent.label_)
  enydisplay = displacy.render(nlp(str(inputtext)), jupyter=True, style='ent') 
  print(enydisplay) 